### <center> ENSF 612 Project </center>
##### <center>Scraping IMDb website </center>

In [13]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 9.3 MB/s eta 0:00:0000:01m0:01
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [14]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import selenium

import warnings
warnings.filterwarnings(action="once")

In [15]:
import certifi
import urllib3

http = urllib3.PoolManager(
    cert_reqs="CERT_REQUIRED",
    ca_certs=certifi.where()
)

http.request("GET", "https://httpbin.org/")

In [16]:
#URL list from IMDB.  Only pages with atleast 20 reviews.  
# After page 351 most movies don't have reviews (and even some on these pages probably don't have enough
url_list = ["https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=51&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=101&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=151&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=201&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=251&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=301&ref_=adv_prv"
            ,"https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start=351&ref_=adv_prv"]



#create list to store movie titles in
movieURLlist = list()

#loop through all the pages and get the end of the hyperlink to each movie.  It usually looks like '/title/tt6710474/' or something like that
for pages in url_list:
    html=requests.get(pages, verify=False).text
    soup = BeautifulSoup(html, 'lxml')
    #print(soup)

    for row in soup.find('div',class_="lister-list").find_all('a',href=re.compile("title")):
    #for row in soup.find('div',class_="lister-item-content").find_all('a',href=re.compile("title")):
        movieURLlist.append(row['href'])
        #print(row['href'])

/Users/samuelsofela/opt/anaconda3/envs/ensf-ml3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.imdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# checking number of movies retrieved
len(movieURLlist)

1204

In [19]:
#cleaning link name by removing the "title" part of the movie link
urllist_regexed = list()

for item in movieURLlist:
    urllist_regexed.append(re.search("^.title.*\/",item).group(0))


In [21]:
#remove all the duplicates
urllist_set = set(urllist_regexed)
urllist_list = list(urllist_set)
urllist_reviews = list()
urllist_main = list()
for item in urllist_list:
    urllist_reviews.append("https://www.imdb.com/" + item + "reviews/?ref_=tt_ov_rt")
    urllist_main.append("https://www.imdb.com/" + item)

***Started using selemium from here to retrieve review***

In [22]:
#installing webdriver for selenium 
#source: https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/,

!pip install webdriver-manager

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [23]:
# importing and installing selenium 3 driver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 8.79M/8.79M [00:00<00:00, 17.2MB/s]
/var/folders/18/yxv48zsx08s_dbth29rh3g2r0000gn/T/ipykernel_64358/2161508394.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


#### Please note that HTML class tags change in IMDb, and the tags used in this program were the most recent as at the time of scraping IMDb website

In [ ]:
from selenium.webdriver.common.by import By

#creating dataframe to store movie data.
df_review=pd.DataFrame(columns=['Movie Title','Movie Rating','Movie Genre','Review Date','Review Title', 'Review Content','Reviewer Rating'])
for movie_page in urllist_main[0:80]:
    #loading movie main page
    driver.get(movie_page)
    #retrieveing movie ratings from main page
    movie_rating = driver.find_element(By.CLASS_NAME, 'sc-e457ee34-1.gvYTvP').text
    #retrieving movie genre from main page
    mov_genre = driver.find_element(By.CLASS_NAME, 'ipc-chip-list__scroller').find_elements(By.CLASS_NAME, 'ipc-chip__text')
    movie_genre = ' '.join([mov_genre[gen].text for gen in range(len(mov_genre))])
    
    

    #Retrieving data from the review page
    driver.get(movie_page+"reviews/?ref_=tt_ov_rt")
    review_count=driver.find_element(By.CLASS_NAME, 'lister').find_element(By.CLASS_NAME, 'header').text
    review_count=review_count.split(" ",2)[0].replace('(','').replace(')','').replace(',','')
    if(int(review_count)<50):
        continue
    driver.implicitly_wait(1)
    
    page = 1
    while page<4: #clicking the 'load more' 4times, which means 100 reviews
        try:
            #the load more button loads additional 25 reviews on the page 
            load_more = driver.find_element(By.ID, 'load-more-trigger')
            load_more.click()
            page +=1
        except:
            break
    review = driver.find_elements(By.CLASS_NAME, 'review-container')
    title_hunt = driver.find_element(By.CLASS_NAME, 'parent').find_element(By.TAG_NAME,'a').text
    #list to store movie reviews
    title=[]
    date=[]
    content = []
    reviewer_rating = []
    num_of_reviews=0
    for i in range (0,50): #using only the first 50 reviews. We can pick more
        try:
            #retrieving title of review
            rev_title=review[i].find_element(By.CLASS_NAME, 'title').text
            #retrieving date of movie review
            rev_date=review[i].find_element(By.CLASS_NAME, 'review-date').text
            #retrieving rating of the review
            rev_rating=review[i].find_element(By.CLASS_NAME, 'ipl-ratings-bar').text.split('/')[0]
            #retrieving review of moview
            rev_content = review[i].find_element(By.CLASS_NAME,'content').find_element(By.CLASS_NAME,'text.show-more__control').text
            if(rev_content==''):
                continue
            #storing retrieved data in appropriate list
            title.append(rev_title)
            date.append(rev_date)
            content.append(rev_content)
            reviewer_rating.append(rev_rating)
            num_of_reviews+=1
        except:
            continue
        #ensures only 25 reviews are selected
        if(num_of_reviews==25):
            break
    #storing the review data of one movie into the dataframe
    one_reviewset = pd.DataFrame({'Movie Title':title_hunt,
                     'Movie Rating':movie_rating,
                     'Movie Genre': movie_genre,
                     'Review Date':date,
                     'Review Title':title,
                     'Review Content':content,
                     'Reviewer Rating':reviewer_rating
               })
    pd_review = pd.DataFrame(one_reviewset)
    #appending review data of all movies
    df_review=df_review.append(one_reviewset, ignore_index=True)


    

In [ ]:
# saving the dataframe into a csv
df_review.to_csv('IMDB_Dataset.csv')
df_review.head(100)